# Classification algorithms

The goal of this notebook is to manipulate some of the classification algorithms of [scikit-learn](http://scikit-learn.org/stable/documentation.html). 

This notebook was created by [Chloé-Agathe Azencott](http://cazencott.info).

This notebook was created using
* python 3.4.3
* numpy 1.15.0
* matplotlib 2.2.2
* scikit-learn 0.19.2

You can check your version of Python by running
```python
import sys
print(sys.version)
```

and the version of any module by running
```python
import <module name>
print(<module name>.__version__)
```

### Loading the data science libraries

In [ ]:
%pylab inline
import pandas as pd

## 1. Predicting mushroom edibility

We will work with a data set that describes mushrooms according to the shape of their cap and stalk, their odor, the type of their veil, etc. This data set also contains information on whether a mushroom is edible or not, and that is what we will try to predict.

Data are available as `data/mushrooms.csv`. Let us load them in a pandas DataFrame called `df`.

In [ ]:
df = pd.read_csv('data/mushrooms.csv', 
                 #index_col=0 # the first column is the IDs of the samples
                )

Let us look at the first few lines of df

In [ ]:
df.head()

## 2. Preparing the data

## 2.1 Converting the features into numerical values

As you can see, the features are encoded as _letters_. Each letter correspond to a category . For example, for the `cap shape` feature, `b` corresponds to a bell cap, `c` to a conical cap, `f` to a flat cap, `k` to a knobbed cap, `s` to a sunken cap, and `x` to a convex cap. For more details about their meaning, you can consult [the documentation of the data set](https://archive.ics.uci.edu/ml/datasets/Mushroom).

In order to work with this data, we need to convert the categorical attributes into numerical values. Here we will simply convert each letter to a number between 0 and the number of categories, using scikit-learn's [preprocessing.LabelEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html).

This encoding is not necessarily the best, as (for example), an algorithm that uses the Euclidean distance will consider that a convex cap (`x` converted to 5) is closer to a sunken cap (`s` converted to 4) than to a conical cap (`c` converted to 1), and the [one-hot encoding](http://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-categorical-features) is a good alternative. However, it has the drawback of increasing the number of features, and of creating correlated features.

In [ ]:
from sklearn.preprocessing import LabelEncoder

labelencoder=LabelEncoder()
for col in df.columns:
    df[col] = labelencoder.fit_transform(df[col])

In [ ]:
df.head()

## 2.2 Split the data into a train set and a test set

Let us first extact the design matrix `X` and the label vector `y` from the data frame.

The first column, `class`, is our target variable. 

In [ ]:
X = df.iloc[:, 1:] # exclude first column
y = df.iloc[:, 0]  # first column only

We will use 70% of the samples of the data for training our models, and the remaining samples to evaluate them.

Let us create the numpy arrays `Xtrain`, `Xtest` and the vectors `ytrain`, `ytest` that we will work with, using scikit-learn's [model_selection.train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function.

In [ ]:
from sklearn import model_selection
Xtrain, Xtest, ytrain, ytest = model_selection.train_test_split(X, y, test_size=0.3)

__Question :__ How many training and test samples do we have? How many features?

In [ ]:
# TODO

__Answer:__

## 2.3 Model selection by cross-validation
Remember, using the test data to select our best model is likely to result in _overfitting_. 

We will therefore start by splitting the training data into 5 folds of cross-validation. Scikit-learn also does that for us.

scikit-learn model selection: http://scikit-learn.org/stable/model_selection.html#model-selection

In [ ]:
# Créate a KFold object
kf = model_selection.KFold(n_splits=5,  # 5 folds
                           shuffle=True # shuffle the samples before splitting 
                          )

# Use kf to split Xtrain into 5 folds. 
# kf.split returns an iterator ((that would be consumed after one iteration). 
# We transform it in a list (on which we can iterate as often as we want).
kf_indices = list(kf.split(Xtrain))

In [ ]:
for i, (tr, te) in enumerate(kf_indices):
    print("Training data for fold %d:" % i, tr)
    print("Test data for fold %d:" % i, te)

# 3. Linear models

We will use scikit-learn to train a few [linear models](http://scikit-learn.org/stable/modules/linear_model.html#linear-model) on `(Xtrain, ytrain)`.

In [ ]:
from sklearn import linear_model

## 3.1 Logistic regression
How does a logistic regression perform on this data? We'll evaluate it by cross-validation, using the cross-validation folds we have generated above.

But wait! What will our criterion be? There are several you can choose from on http://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter. In this lab, we will use the [F1 score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score).

__Question :__ What other criterion or criteria could you use?

__Annswer:__ 

Let us create a logistic regression model. We will use [scikit-learn's implementation](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression), which includes a regularization parameter.

__Question:__ What is the relationship between the regularization parameter `C` and the regularization parameter `alpha` of the [ridge regression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge) ?

__Answer:__

In [ ]:
model = linear_model.LogisticRegression(C=1e10) # use a large penalty 
                                                # so as not to regularize

scikit-learn allows us to directly compute the cross-validated F1 score (on the train test) of this model:

In [ ]:
f1 = model_selection.cross_val_score(model, 
                                     Xtrain, y=ytrain, 
                                     scoring='f1', 
                                     cv=kf_indices)

print(["%.3f" % value for value in f1])

__Question:__ What is the average f1 score of the linear regression over all folds?

In [ ]:
# TODO

What are the weights that a logistic regression assigns to each variable on the train set? To determine this, we will train a logistic regression on the entire training data.

In [ ]:
# Train a logistic regression on the entire train set
model.fit(Xtrain, ytrain)

# Create a figure
fig = plt.figure(figsize(12, 6))

# Plot, for each feature, its coefficient in the model
num_features = Xtrain.shape[1]
plt.scatter(range(num_features), model.coef_)

plt.xlabel('Feature', fontsize=14)
tmp = plt.xticks(range(num_features), list(df.columns[1:]), 
                 rotation=90, fontsize=14)
tmp = plt.ylabel('Weight', fontsize=14)

tmp = plt.title('Logistic regression weights', fontsize=16)

__Question:__ Which feature do you think is the most important to predict edibility? Which one is the least important?

In [ ]:
# TODO

__Answer:__

## 3.2 Ridge logistic regression

Can we improve performance using regularization? By default, the logistic regression implemented in scikit-learn uses a l2 (i.e. ridge) penalty. Let us now try to find the optimal value of the regularization parameter `C`.

Let us create 50 values of `C` for testing, equally spaced (in log scale) between $10^{-1}$ and $10^4$:

In [ ]:
cvalues = np.logspace(-1, 4, 50)

f1_per_c = [] # will store the f1 values for all 50 values of C
weights_per_c = [] #  will store the weights associated with each feature 
                       # for all 50 values of C
for cval in cvalues:
    # Create a logistic regression ridge model
    model = linear_model.LogisticRegression(C=cval)
    
    # Compute the model's cross-validated performance
    f1 = model_selection.cross_val_score(model, 
                                           Xtrain, y=ytrain, 
                                           scoring='f1', 
                                           cv=kf_indices)
    f1_per_c.append(f1)
    
    # Train the model on the entire data set and store the regression coefficients
    model.fit(Xtrain, ytrain)
    weights_per_c.append(model.coef_)

In [ ]:
# Reshape weights_per_c into a (number of values of C, number of features) array
weights_per_c = np.array(weights_per_c)
weights_per_c.shape = (50, X.shape[1])

In [ ]:
# Plot, for each feature, the regression weight as a function of alpha
fig = plt.figure(figsize=(8, 8))

lines = plt.plot(cvalues, weights_per_c)
plt.xscale('log')
tmp = plt.legend(lines, list(df.columns[1:]), frameon=False, 
                 loc=(1, 0), fontsize=14)

tmp = plt.xlabel('C', fontsize=14)
tmp = plt.ylabel('Weight', fontsize=14)

tmp = plt.title('Logistic ridge regression weights', fontsize=16)

__Question:__ Is this consistent with what we observed for the non-regularized logistic regression? 

__Answer:__

__Question:__ Plot the f1 score (averaged across the 5 folds) as a function of C.

In [ ]:
# TODO


__Question:__ What is the optimal value of C and what is the corresponding average cross-validated f1? Use `np.argmin`.

In [ ]:
# TODO


__Answer:__

__Question:__ How does the regularized logistic ridge regression compare to the vanilla logistic regression?

__Answer:__

# 4. Non-linear models
Can a non-linear model improve the performance on this data? Our performance are already very good!

# 5. Final model

__Question:__ Based on the above analyses, which model do you finally choose as the most performant? What is its confusion matrix on the test set? 

In [ ]:
# TODO

__Answer:__